# Visual Area Segmentation

##### copied/adapted from:

https://github.com/zhuangjun1981/NeuroAnalysisTools/blob/master/NeuroAnalysisTools/RetinotopicMapping.py

##### Cite the original work/implementation:

[Zhuang et al., Elife (2017)](https://elifesciences.org/articles/18372)

In [ ]:
# load packages:
import numpy as np
import sys
sys.path.append('../src')
import physion.utils.plot_tools as pt
from physion.intrinsic.tools import *
from physion.intrinsic.analysis import RetinotopicMapping
import matplotlib.pylab as plt
from PIL import Image

## Load data

In [ ]:
dataFolder = '/Users/yann/UNPROCESSED/CIBELE/2024_06_28/12-36-46/'
# retinotopic mapping data
maps = np.load(os.path.join(dataFolder, 'raw-maps.npy'), allow_pickle=True).item()
# vasculature picture
imVasc = np.array(Image.open(os.path.join(dataFolder, 'vasculature-PVNR1_M3-HQ.tif')))
plt.imshow(imVasc, cmap=plt.cm.grey); plt.axis('off');

# Retinotopic Maps

In [ ]:
fig, AX = pt.figure(axes=(2,2), figsize=(1.5,2), hspace=0.5, wspace=0.5)

for p, direction in enumerate(['right', 'left']):
    plot_power_map(AX[0][p], fig, maps['%s-power' % direction])
    plot_phase_map(AX[1][p], fig, maps['%s-phase' % direction])
    AX[0][p].set_title(direction)
for ax in pt.flatten(AX):
    ax.axis('off')

In [ ]:
fig, AX = pt.figure(axes=(2,2), figsize=(1.5,2), hspace=0.5, wspace=0.5)

for p, direction in enumerate(['up', 'down']):
    plot_power_map(AX[0][p], fig, maps['%s-power' % direction])
    plot_phase_map(AX[1][p], fig, maps['%s-phase' % direction])
    AX[0][p].set_title(direction)
for ax in pt.flatten(AX):
    ax.axis('off')

In [ ]:
plot_retinotopic_maps(maps, map_type='altitude');

In [ ]:
plot_retinotopic_maps(maps, map_type='azimuth');

# Perform Segmentation

In [ ]:
data = build_trial_data(maps)
segmentation_params={'phaseMapFilterSigma': 2.,
                     'signMapFilterSigma': 2.,
                     'signMapThr': 0.65,
                     'eccMapFilterSigma': 10.,
                     'splitLocalMinCutStep': 5.,
                     'mergeOverlapThr': 0.1,
                     'closeIter': 3,
                     'openIter': 3,
                     'dilationIter': 15,
                     'borderWidth': 1,
                     'smallPatchThr': 100,
                     'visualSpacePixelSize': 0.5,
                     'visualSpaceCloseIter': 15,
                     'splitOverlapThr': 1.1}
data['params'] = segmentation_params
trial = RetinotopicMapping.RetinotopicMappingTrial(**data)
trial.processTrial(isPlot=True)

In [ ]:
fig, ax = pt.figure(figsize=(5,5))
h = RetinotopicMapping.plotPatches(trial.finalPatches, plotaxis=ax)
ax.imshow(imVasc, cmap=plt.cm.grey, vmin=imVasc.min(), vmax=imVasc.max(), extent=[*ax.get_xlim(), *ax.get_ylim()])
h = RetinotopicMapping.plotPatches(trial.finalPatches, plotaxis=ax)
ax.axis('off');
#fig.savefig(os.path.join(os.path.expanduser('~'), 'Desktop', 'fig.svg'))